# Pattern Matching for Anti-Money Laundering

In [ ]:
import pandas as pd
import swat

In [ ]:
host = 'orgrd015.unx.sas.com' 
port =  12776
username = 'sthare'
password = ''
s = swat.CAS(host, port, username, password)

In [ ]:
s.loadActionSet('Network')
s.loadActionSet('fcmpact')
s.sessionProp.setSessOpt(cmplib = "casuser.myRoutines")

## Input data

In [ ]:
nodes = [['A', 2015], ['B', 2014], ['C', 2016], ['D', 2016], ['E', 2016], ['F', 2017]]
links = [['A', 'B', '02OCT2017'],
             ['A', 'C', '03OCT2017'],
             ['B', 'C', '03OCT2017'],
             ['B', 'D', '04OCT2017'],
             ['C', 'A', '02OCT2017'],
             ['C', 'D', '04OCT2017'],
             ['D', 'A', '01NOV2017'],
             ['D', 'E', '01NOV2017'],
             ['D', 'F', '17NOV2017'],
             ['E', 'B', '04OCT2017'],
             ['F', 'B', '13FEB2018'],
             ['F', 'E', '13FEB2018']]

In [ ]:
nodes = pd.DataFrame(nodes, columns = ['node', 'year'])
links = pd.DataFrame(links, columns = ['from', 'to', 'time'])
s.upload(nodes, casout = {'name':'nodes', 'replace':True})
s.upload(links, casout = {'name':'links', 'replace':True},
            importOptions = {"vars": [{"name":"from", "type": "VARCHAR"},
                                      {"name":"to"  , "type": "VARCHAR"},
                                      {"name":"time", "type": "Date", "informat": "date9.", "format": "date9."}]})


## Query

In [ ]:
nodesQuery = [1, 2, 3]
linksQuery = [[1, 2], [2, 3], [3, 1]]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to'])

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})

In [ ]:
s.addRoutines(
    routineCode = '''
    function myNodePairFilter(nodeQ[*], year[*]); 
        if (nodeQ[1] = 2 and nodeQ[2] = 3) then return (year[1] = year[2]); 
        else return (1); 
    endsub;
    function myLinkPairFilter(fromQ[*], toQ[*], time[*]);
        if (toQ[1] = 1) then return (1);
        else if (toQ[1] = fromQ[2]) then return (time[1] < time[2]); 
             else return (1);
    endsub;         
    ''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name":"myRoutines", "replace":True}
)  

In [ ]:
s.network.patternMatch(
    direction      = "directed",
    nodes          = {"name":"nodes"},
    links          = {"name":"links"},
    nodesQuery     = {"name":"nodesQuery"},
    linksQuery     = {"name":"linksQuery"},
    nodesVar       = {"vars":"year"},
    linksVar       = {"vars":"time"},
    nodePairFilter = "myNodePairFilter",
    linkPairFilter = "myLinkPairFilter",
    outMatchNodes  = {"name":"OutMatchNodes", "replace":True},
    outMatchLinks  = {"name":"OutMatchLinks", "replace":True} 
)

In [ ]:
print(s.CASTable("OutMatchNodes").to_frame())
print(s.CASTable("OutMatchLinks").to_frame())